In [1]:
print("Hello")

Hello


# ❓ Q&A Over SQL Data

## 🧩 Introduction

We will build a Question & Answer (Q&A) application that interacts with **tabular data stored in SQL databases**.

### 🗣️ Natural Language Interface
This app will allow users to:
- Ask questions about the database content **using natural language**.
- Receive answers **also in natural language**, generated by the model.

### ⚠️ Security Note
Building Q&A systems over SQL databases involves executing **model-generated SQL queries**, which can pose security risks.

> **In production**, always ensure that your **database connection permissions are scoped as narrowly as possible**, based on the specific requirements of your application chain.


In [2]:
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


from dotenv import load_dotenv, find_dotenv

In [3]:
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model= "llama-3.3-70b-versatile",
    temperature= 0.1
)

## 🔌 Connecting to the Database

In this example, we will use a **SQLite** connection to interact with the `street_tree_db` database located in the `data` folder.

### 📥 Prerequisite
> **Make sure to download the `data` folder from the GitHub repository** and place it in the **root directory** of this project.

### 🧩 Database Interface
We will interact with the database using the **`SQLDatabase`** class provided by **SQLAlchemy**.


In [ ]:
from langchain_community.utilities import SQLDatabase

sql_db_path = r"Data\street_tree_db.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{sql_db_path}")

## 🔄 Creating the SQL Query Chain

We can build a simple chain that processes a user's question and performs the following steps:

1. **Convert** the natural language question into a SQL query.
2. **Execute** the generated SQL query.
3. **Use** the query result to answer the original question.

### 🧠 Built-in Support with LangChain

The first step in an SQL chain is to take the user’s input and translate it into a SQL query.  
LangChain provides a built-in chain for this purpose called `create_sql_query_chain`.

---

### 🪄 Step 1: Translating a Natural Language Question into an SQL Query

In [6]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many species of trees are in san Francisco?"})

In [11]:
import re

type(response), response
match = re.search(r'SQLQuery:\s*(.+)', response)

if match:
    sql_query = match.group(1).strip()
    print("Extracted SQL query:", sql_query)
else:
    print("SQL query not found in the response.")


Extracted SQL query: SELECT COUNT(DISTINCT "qSpecies") FROM street_trees


In [12]:
db.run(sql_query)

'[(148,)]'

In [13]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

## 🧾 Step 2: Executing the SQL Query

Now that we've generated a SQL query, the next step is to **execute it**.

### ⚠️ Important Note for Production
Before running automated queries in a production environment, carefully assess the risks:
- Limit database permissions as much as possible.
- Consider adding a **human approval step** before executing model-generated queries.

### ⚙️ Executing with `QuerySQLDatabaseTool`

We can use the **`QuerySQLDatabaseTool`** to easily add query execution capabilities to our LangChain pipeline.

#### 🔄 How the Chain Works:
- The **user asks a question**.
- The **write_query chain** takes the question as input and generates the SQL query.
- The **execute_query chain** takes the SQL query as input and runs it, returning the result.


In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "List the species of trees thar are present in san Francisco?"})
## Dont work check later

'Error: (sqlite3.OperationalError) near "Question": syntax error\n[SQL: Question: List the species of trees that are present in San Francisco?\nSQLQuery: SELECT DISTINCT "qSpecies" FROM street_trees ORDER BY "qSpecies" LIMIT 5]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

## 🗣️ Step 3: Translating the SQL Response into Natural Language

Now that we can generate and execute SQL queries, the final step is to **combine the original question with the query result** and use a chat model to produce a **natural language answer**.

### 🧠 How it Works
We pass both:
- the **original user question**, and
- the **SQL query result**

to the LLM (Large Language Model), which then generates a **final, human-readable answer**.

This helps create an intuitive user experience by returning responses in natural language instead of raw SQL data.


In [16]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SQLDatabase

sql_db_path = r"Data\street_tree_db.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{sql_db_path}")

answer_template = """
    
    Given the following user question,
    corresponding SQL query, and SQL result,
    answer the user question.

    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """

answer_prompt = PromptTemplate.from_template(answer_template)

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

In [18]:
chain  = (
    RunnablePassthrough.assign(query=write_query).assign(
        result = itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "List the species of trees thar are present in San Francisco?"})

'To answer the user\'s question, we need to correct the SQL query. The error message indicates a syntax error near the word "Question", which suggests that the query is not properly formatted.\n\nThe corrected SQL query should be:\n\n```sql\nSELECT DISTINCT "qSpecies" \nFROM street_trees \nWHERE "qAddress" LIKE \'%San Francisco%\' \nORDER BY "qSpecies";\n```\n\nThis query will return the distinct species of trees present in San Francisco. The `WHERE` clause filters the results to only include trees located in San Francisco, and the `ORDER BY` clause sorts the results alphabetically by species.\n\nNote: The `LIMIT 5` clause in the original query would only return the first 5 species, which may not be the desired result. I have removed it to return all species present in San Francisco.\n\nAssuming the corrected query returns a result set, the answer to the user\'s question would be a list of tree species present in San Francisco, such as:\n\n* Redwood\n* Oak\n* Pine\n* Maple\n* Cypress\n